# AgentV1

> Fill in a module description here

In [ ]:
#| default_exp AgentV1

In [ ]:
#|export
from KhazadDum.SnowflakeCore import *
from lisette import *
import pandas as pd
import ast
import regex as re
import json

In [ ]:
#|export
from IPython.display import Markdown, display
MD = lambda x: display(Markdown(x))

## System Prompt

In [ ]:
#|export
def create_system_prompt(schema: ParentSchema) -> str:
    return f"""You are a SQL query generator for {schema.dialect}.

DATABASE: {schema.database}.{schema.Schema}
SCHEMA: {schema.model_dump_json(indent=2)}

CORE RULES:
1. Generate SELECT queries only - no modifications
2. ALWAYS quote identifiers: SELECT "column_name" FROM "table_name"
3. Use fully qualified names: {schema.database}.{schema.Schema}."TABLE_NAME"
4. No semicolons - single statement only
5. Use the execute_query tool to run queries
6. Format queries in triple quotes for readability
7. If user asks for all data do not apply `limit`

JSON COLUMNS:
- Many columns store JSON as VARCHAR (see samples)
- To extract values: GET(PARSE_JSON("column_name"), 'key')::VARCHAR
- Example: GET(PARSE_JSON("airport_name"), 'en')::VARCHAR

NULL VALUES:
- Sample data shows '\\N' but use IS NULL / IS NOT NULL in WHERE clauses

JSON COLUMNS:
- Many columns store JSON as VARCHAR (city, airport_name, model)
- Extract English values: GET(PARSE_JSON("column_name"), 'en')::VARCHAR
- Example: GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name"

ERROR RECOVERY:
If query fails:
1. Check quoted identifiers (all table/column names)
2. Verify table names are fully qualified
3. Confirm JSON extraction syntax
4. Check for typos in column/table names
5. Retry with corrected query

COMMON PATTERNS:
- Aggregations: Use COUNT(*), SUM(), AVG() with GROUP BY
- Top N: Use ORDER BY ... LIMIT N
- Deduplication: Use DISTINCT or GROUP BY
- Date filtering: Use BETWEEN or comparison operators on date columns

PERFORMANCE:
- Always use LIMIT for exploratory queries
- Prefer specific columns over SELECT *
- Use indexes via foreign key relationships

Be precise, efficient, and always validate your SQL syntax before execution."""


## Experiment

In [ ]:
agent = SnowflakeAgent()

M = DBMetadata(
    agent,
    "AIRLINES", "AIRLINES", model_name = model_name)
assert M()

In [ ]:
SYSTEM_PROMPT = create_system_prompt(M.metadata)
print(SYSTEM_PROMPT)

You are a SQL query generator for snowflake.

DATABASE: AIRLINES.AIRLINES
SCHEMA: {
  "dialect": "snowflake",
  "database": "AIRLINES",
  "Schema": "AIRLINES",
  "tables": [
    {
      "name": "AIRLINES.AIRLINES.AIRCRAFTS_DATA",
      "column_names": [
        {
          "name": "aircraft_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "model",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "range",
          "type": "NUMBER(38,0)",
          "null?": "Y"
        }
      ],
      "sample_rows": [
        {
          "aircraft_code": "773",
          "model": "{\"en\": \"Boeing 777-300\", \"ru\": \"Боинг 777-300\"}",
          "range": 11100
        }
      ],
      "row_count": 9
    },
    {
      "name": "AIRLINES.AIRLINES.AIRPORTS_DATA",
      "column_names": [
        {
          "name": "airport_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
   

In [ ]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("Which airport has the most departures?", max_steps=5)
res

The airport with the most departures is **Domodedovo International Airport** (airport code DME), with **3,217** scheduled departures in the dataset.

<details>

- id: `chatcmpl-1ooxlmvdl85k80r5m4f1qc`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=48, prompt_tokens=3352, total_tokens=3400, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
chat.print_hist()

{'role': 'user', 'content': 'Which airport has the most departures?'}

Message(content='', role='assistant', tool_calls=[{'function': {'arguments': '{"query":"SELECT GET(PARSE_JSON(\\"airport_name\\"), \'en\')::VARCHAR AS \\"airport_name\\", COUNT(*) AS \\"departure_count\\"\\nFROM AIRLINES.AIRLINES.\\"FLIGHTS\\"\\nJOIN AIRLINES.AIRLINES.\\"AIRPORTS_DATA\\" ON \\"FLIGHTS\\".\\"departure_airport\\" = \\"AIRPORTS_DATA\\".\\"airport_code\\"\\nGROUP BY \\"airport_name\\"\\nORDER BY \\"departure_count\\" DESC\\nLIMIT 1","max_rows":10}', 'name': 'execute_query'}, 'id': '477437910', 'type': 'function'}], function_call=None, reasoning_content='Need count of flights per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport. Order desc limit 1. Extract airport_name en. Need join to AIRPORTS_DATA. Query.', provider_specific_fields={'refusal': None, 'reasoning': 'Need count of flights per departure_airport. Use FLIGHTS table

In [ ]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("Which airport has the least departures?", max_steps=5)
res

In [ ]:
chat.print_hist()

{'role': 'user', 'content': 'Which airport has the least departures?'}

Message(content='', role='assistant', tool_calls=[{'function': {'arguments': '{"query":"SELECT \\"departure_airport\\" AS \\"airport_code\\", COUNT(*) AS \\"departures_count\\", GET(PARSE_JSON(\\"airport_name\\"), \'en\')::VARCHAR AS \\"airport_name\\"\\nFROM AIRLINES.AIRLINES.FLIGHTS\\nJOIN AIRLINES.AIRLINES.AIRPORTS_DATA ON FLIGHTS.\\"departure_airport\\" = AIRPORTS_DATA.\\"airport_code\\"\\nGROUP BY \\"departure_airport\\", GET(PARSE_JSON(\\"airport_name\\"), \'en\')::VARCHAR\\nORDER BY \\"departures_count\\" ASC\\nLIMIT 1","max_rows":10}', 'name': 'execute_query'}, 'id': '298743786', 'type': 'function'}], function_call=None, reasoning_content='Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Use GET(PARSE_JSON("airport_name")

## For formtting user input


In [ ]:
chat.hist[0]

{'role': 'user', 'content': 'Which airport has the least departures?'}

In [ ]:
#|export
def md_user(msg):
    """Format user message with quote styling"""
    content = msg.get('content', '') if isinstance(msg, dict) else msg.content
    
    return f"""### 👤 **User**

> {content}
"""

In [ ]:
MD(md_user(chat.hist[0]))

### 👤 **User**

> Which airport has the least departures?


## For formtting  tool execute query response

Implement rendering of chat  history for better visualization and analysis.

In [ ]:
content = chat.hist[2]['content']

# Extract the key-value pairs
# This regex finds pattern: key=value
pattern = r"(\w+)=(.+?)(?=\s+\w+=|\s*$)"
matches = re.findall(pattern, content)
matches

[('query',
  '\'SELECT "departure_airport" AS "airport_code", COUNT(*) AS "departures_count", GET(PARSE_JSON("airport_name"), \\\'en\\\')::VARCHAR AS "airport_name"\\nFROM AIRLINES.AIRLINES.FLIGHTS\\nJOIN AIRLINES.AIRLINES.AIRPORTS_DATA ON FLIGHTS."departure_airport" = AIRPORTS_DATA."airport_code"\\nGROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), \\\'en\\\')::VARCHAR\\nORDER BY "departures_count" ASC\\nLIMIT 1\''),
 ('success', 'True'),
 ('data',
  "[{'airport_code': 'USK', 'departures_count': 18, 'airport_name': 'Usinsk Airport'}]"),
 ('error', 'None'),
 ('row_count', '1'),
 ('execution_time', '0.38482069969177246')]

In [ ]:
ast.literal_eval(matches[2][1].strip())

[{'airport_code': 'USK',
  'departures_count': 18,
  'airport_name': 'Usinsk Airport'}]

In [ ]:
chat.hist[2]

{'tool_call_id': '298743786',
 'role': 'tool',
 'name': 'execute_query',
 'content': 'query=\'SELECT "departure_airport" AS "airport_code", COUNT(*) AS "departures_count", GET(PARSE_JSON("airport_name"), \\\'en\\\')::VARCHAR AS "airport_name"\\nFROM AIRLINES.AIRLINES.FLIGHTS\\nJOIN AIRLINES.AIRLINES.AIRPORTS_DATA ON FLIGHTS."departure_airport" = AIRPORTS_DATA."airport_code"\\nGROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), \\\'en\\\')::VARCHAR\\nORDER BY "departures_count" ASC\\nLIMIT 1\' success=True data=[{\'airport_code\': \'USK\', \'departures_count\': 18, \'airport_name\': \'Usinsk Airport\'}] error=None row_count=1 execution_time=0.38482069969177246'}

We want the details of content. It is a string object.

In [ ]:
#|export
def str2key_value(msg:str):
    """
    Extracts key and value from a message string.

    Args:
        msg (str): The message string to extract tools from.
        Ex: `'query=\'SELECT "departure_airport", COUNT(*) AS "departures" FROM AIRLINES.AIRLINES."FLIGHTS" WHERE "scheduled_departure" IS NOT NULL GROUP BY "departure_airport" ORDER BY "departures" ASC LIMIT 1\' success=True data=[{\'departure_airport\': \'USK\', \'departures\': 18}] error=None row_count=1 execution_time=0.3334343433380127'`

    Returns:
        dict: A dictionary of tools extracted from the message string.
        Ex:
        {'query': 'SELECT "departure_airport", COUNT(*) AS "departures" FROM AIRLINES.AIRLINES."FLIGHTS" WHERE "scheduled_departure" IS NOT NULL GROUP BY "departure_airport" ORDER BY "departures" ASC LIMIT 1',
        'success': True,
        'data': [{'departure_airport': 'USK', 'departures': 18}],
        'error': None,
        'row_count': 1,
        'execution_time': 0.3334343433380127}

    """
    pattern = r"(\w+)=(.+?)(?=\s+\w+=|\s*$)"
    matches = re.findall(pattern, msg)
    parsed = {}
    for key, value in matches:
        try:
            # Try to evaluate as Python literal
            parsed[key] = ast.literal_eval(value.strip())
        except (ValueError, SyntaxError):
            # If it fails, keep as string
            parsed[key] = value.strip()
    return parsed

In [ ]:
str2key_value('content')

{}

In [ ]:
dic = str2key_value(chat.hist[2]['content'])
dic

{'query': 'SELECT "departure_airport" AS "airport_code", COUNT(*) AS "departures_count", GET(PARSE_JSON("airport_name"), \'en\')::VARCHAR AS "airport_name"\nFROM AIRLINES.AIRLINES.FLIGHTS\nJOIN AIRLINES.AIRLINES.AIRPORTS_DATA ON FLIGHTS."departure_airport" = AIRPORTS_DATA."airport_code"\nGROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), \'en\')::VARCHAR\nORDER BY "departures_count" ASC\nLIMIT 1',
 'success': True,
 'data': [{'airport_code': 'USK',
   'departures_count': 18,
   'airport_name': 'Usinsk Airport'}],
 'error': None,
 'row_count': 1,
 'execution_time': 0.38482069969177246}

In [ ]:
#|export
def md_tool(dic):
    """Compact tool result formatting"""
    content = str2key_value(dic.get('content', ''))
    
    success = content.get('success', False)
    data = content.get('data', [])
    row_count = content.get('row_count', 0)
    exec_time = content.get('execution_time', 0)
    
    base_str = f"""### 🛠️ **Tool**
__Executed: {dic.get('name', '')}__

"""
    
    if not success:
        return base_str + f"❌ **Error:** {content.get('error')}"
    
    # Format first few rows
    if data: 
        if isinstance(data, str):
            # rather than conerting str to table
            # better to refire the query ti db 
            query = content.get('query', '')
            r = agent.execute_query(query)
            df = pd.json_normalize(r.data)
        else:
            df = pd.json_normalize(data[:5])
        
        table = df.to_markdown(index=False)
    else:
        table = "_No data_"
    
    return base_str + f"""✅ **Result** ({row_count} rows, {exec_time:.2f}s)

{table}
"""

In [ ]:
MD(md_tool(chat.hist[2]))

### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.38s)

| airport_code   |   departures_count | airport_name   |
|:---------------|-------------------:|:---------------|
| USK            |                 18 | Usinsk Airport |


## For formtting  assistant response

In [ ]:
#|export
from litellm.types.utils import Message
def md_assistant(msg:Message):
    """Format message as markdown - handles both tool calls and regular responses"""
    
    # Check if this is a tool call message
    if msg.tool_calls and len(msg.tool_calls) > 0:
        # Format tool call
        try:
            func = msg.tool_calls[0]['function']
            args = json.loads(func['arguments'])
            query = args.pop('query', '')
            
            query_section = f"""<details>
<summary><b>📝 Query</b></summary>

```sql
{query}

```
</details>
"""  if query else ''
            #query_section = f">**Query:**\n```sql\n{query}\n```\n" if query else ''
            
            args_table = (
                "\n| Parameter | Value |\n|-----------|-------|\n" + 
                "\n".join(f"| **{k}** | `{v}` |" for k, v in args.items())
            ) if args else ""
            
            reasoning = msg.reasoning_content or "No reasoning provided"
            
            return f"""### 🧠 Assistant \n\n
_Reasoning: {reasoning}_

**Call: {func['name']}**

{query_section}{args_table}

"""
        except (IndexError, KeyError, json.JSONDecodeError) as e:
            return f"❌ Error formatting tool call: {e}"
    
    else:
        # Regular assistant message (no tool calls)
        reasoning = msg.reasoning_content or ""
        reasoning_section = f"\n\n💭 _Reasoning: {reasoning}_" if reasoning else ""
        
        return f"""🤖 **Assistant**

{msg.content}{reasoning_section}
"""

In [ ]:
m = md_assistant(chat.hist[1])
display(Markdown(m))

### 🧠 Assistant 


_Reasoning: Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Use GET(PARSE_JSON("airport_name"),'en')::VARCHAR AS "airport_name". Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT "departure_airport" AS "airport_code", COUNT(*) AS "departures_count", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name"
FROM AIRLINES.AIRLINES.FLIGHTS
JOIN AIRLINES.AIRLINES.AIRPORTS_DATA ON FLIGHTS."departure_airport" = AIRPORTS_DATA."airport_code"
GROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR
ORDER BY "departures_count" ASC
LIMIT 1

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |



In [ ]:
MD(md_assistant(chat.hist[-1]))

🤖 **Assistant**

The airport with the fewest departures is **USK (Usinsk Airport)**, which has 18 scheduled departures in the dataset.

💭 _Reasoning: Need to provide answer._


## Putting it all together

In [ ]:
r = []
for h in chat.hist:
    if isinstance(h, dict) :
        ## it will be for user and tools
        if h['role'] == 'user':
            r.append(md_user(h))
        elif h['role'] == 'tool':
            r.append(md_tool(h))
    elif isinstance(h, Message):
        ## assitant turn
        r.append(md_assistant(h))

In [ ]:
MD('\n ---\n'.join(r))

### 👤 **User**

> Which airport has the least departures?

 ---
### 🧠 Assistant 


_Reasoning: Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Use GET(PARSE_JSON("airport_name"),'en')::VARCHAR AS "airport_name". Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT "departure_airport" AS "airport_code", COUNT(*) AS "departures_count", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name"
FROM AIRLINES.AIRLINES.FLIGHTS
JOIN AIRLINES.AIRLINES.AIRPORTS_DATA ON FLIGHTS."departure_airport" = AIRPORTS_DATA."airport_code"
GROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR
ORDER BY "departures_count" ASC
LIMIT 1

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.38s)

| airport_code   |   departures_count | airport_name   |
|:---------------|-------------------:|:---------------|
| USK            |                 18 | Usinsk Airport |

 ---
🤖 **Assistant**

The airport with the fewest departures is **USK (Usinsk Airport)**, which has 18 scheduled departures in the dataset.

💭 _Reasoning: Need to provide answer._


In [ ]:
def vis(chat):
        r = []
        for h in chat.hist:
            if isinstance(h, dict) :
                ## it will be for user and tools
                if h['role'] == 'user':
                    r.append(md_user(h))
                elif h['role'] == 'tool':
                    r.append(md_tool(h))
            elif isinstance(h, Message):
                ## assitant turn
                r.append(md_assistant(h))
        return MD('\n ---\n'.join(r))

In [ ]:
vis(chat)

### 👤 **User**

> Which airport has the least departures?

 ---
### 🧠 Assistant 


_Reasoning: Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Use GET(PARSE_JSON("airport_name"),'en')::VARCHAR AS "airport_name". Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT "departure_airport" AS "airport_code", COUNT(*) AS "departures_count", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name"
FROM AIRLINES.AIRLINES.FLIGHTS
JOIN AIRLINES.AIRLINES.AIRPORTS_DATA ON FLIGHTS."departure_airport" = AIRPORTS_DATA."airport_code"
GROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR
ORDER BY "departures_count" ASC
LIMIT 1

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.38s)

| airport_code   |   departures_count | airport_name   |
|:---------------|-------------------:|:---------------|
| USK            |                 18 | Usinsk Airport |

 ---
🤖 **Assistant**

The airport with the fewest departures is **USK (Usinsk Airport)**, which has 18 scheduled departures in the dataset.

💭 _Reasoning: Need to provide answer._


## Other Expriments

### 1.

In [ ]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("How many flights are there?", max_steps=5)
res

The total number of flights in the dataset is **33,121**.

<details>

- id: `chatcmpl-9puc247sf5tgf60o56s0c`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=28, prompt_tokens=3195, total_tokens=3223, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
vis(chat)

### 👤 **User**

> How many flights are there?

 ---
### 🧠 Assistant 


_Reasoning: Need count from FLIGHTS. Use fully qualified._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES."FLIGHTS"

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.36s)

|   flight_count |
|---------------:|
|          33121 |

 ---
🤖 **Assistant**

The total number of flights in the dataset is **33,121**.

💭 _Reasoning: Return result._


### 2

In [ ]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("Which airport has the most departures?", max_steps=5)
res

The airport with the highest number of departures is **Domodedovo International Airport** (airport code DME), with **3,217** scheduled departures in the dataset.

<details>

- id: `chatcmpl-hmbsrxiioht3yaddxlpzf`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=50, prompt_tokens=3352, total_tokens=3402, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
vis(chat)

### 👤 **User**

> Which airport has the most departures?

 ---
### 🧠 Assistant 


_Reasoning: Need count of flights per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport. Order desc limit 1. Extract airport_name en. Need join to AIRPORTS_DATA. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
JOIN AIRLINES.AIRLINES."AIRPORTS_DATA" ON "FLIGHTS"."departure_airport" = "AIRPORTS_DATA"."airport_code"
GROUP BY "airport_name"
ORDER BY "departure_count" DESC
LIMIT 1

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.42s)

| airport_name                     |   departure_count |
|:---------------------------------|------------------:|
| Domodedovo International Airport |              3217 |

 ---
🤖 **Assistant**

The airport with the highest number of departures is **Domodedovo International Airport** (airport code DME), with **3,217** scheduled departures in the dataset.

💭 _Reasoning: Need to provide answer._


### 3

In [ ]:
def ask(q):
    chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
    res = chat(q, max_steps=15)
    return chat, res

In [ ]:
q = "What are the top 5 busiest routes by number of flights?"

chat, res = ask(q)
res

**Top 5 busiest routes (by number of flights)**  

| # | Departure Airport | Arrival Airport | Flights |
|---|--------------------|-----------------|---------|
| 1 | SVO | LED | 305 |
| 2 | LED | SVO | 305 |
| 3 | DME | LED | 244 |
| 4 | LED | DME | 244 |
| 5 | BZK | SVO | 183 |

These counts are derived from the `FLIGHTS` table, grouped by departure and arrival airport codes.

<details>

- id: `chatcmpl-6wqwxyztkoee4ttw8fflin`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=131, prompt_tokens=3376, total_tokens=3507, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
vis(chat)

### 👤 **User**

> What are the top 5 busiest routes by number of flights?

 ---
### 🧠 Assistant 


_Reasoning: Need join FLIGHTS with departure_airport and arrival_airport counts. Count distinct flight_id per route pair. Order desc limit 5. Use fully qualified names. Use SELECT "departure_airport","arrival_airport",COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES.FLIGHTS GROUP BY "departure_airport","arrival_airport" ORDER BY "flight_count" DESC LIMIT 5._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT "departure_airport", "arrival_airport", COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES.FLIGHTS GROUP BY "departure_airport", "arrival_airport" ORDER BY "flight_count" DESC LIMIT 5

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (5 rows, 0.64s)

| departure_airport   | arrival_airport   |   flight_count |
|:--------------------|:------------------|---------------:|
| SVO                 | LED               |            305 |
| LED                 | SVO               |            305 |
| DME                 | LED               |            244 |
| LED                 | DME               |            244 |
| BZK                 | SVO               |            183 |

 ---
🤖 **Assistant**

**Top 5 busiest routes (by number of flights)**  

| # | Departure Airport | Arrival Airport | Flights |
|---|--------------------|-----------------|---------|
| 1 | SVO | LED | 305 |
| 2 | LED | SVO | 305 |
| 3 | DME | LED | 244 |
| 4 | LED | DME | 244 |
| 5 | BZK | SVO | 183 |

These counts are derived from the `FLIGHTS` table, grouped by departure and arrival airport codes.

💭 _Reasoning: Need to format answer._


### 4

In [ ]:
q = "Which aircraft models have the most seats available?"

chat, res = ask(q)
res

The aircraft models with the most seats available (based on the `SEATS` table) are:

| Model | Seats |
|-------|-------|
| Boeing 777‑300 | 402 |
| Boeing 767‑300 | 222 |
| Airbus A321‑200 | 170 |
| Airbus A320‑200 | 140 |
| Boeing 737‑300 | 130 |
| Airbus A319‑100 | 116 |
| Sukhoi Superjet‑100 | 97 |
| Bombardier CRJ‑200 | 50 |
| Cessna 208 Caravan | 12 |

These counts represent the total number of seat rows recorded for each aircraft model in the dataset.

<details>

- id: `chatcmpl-kbcoj1z1s8fq7jt45uh0r`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=158, prompt_tokens=3488, total_tokens=3646, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
vis(chat)

### 👤 **User**

> Which aircraft models have the most seats available?

 ---
### 🧠 Assistant 


_Reasoning: Need join SEATS with AIRCRAFTS_DATA to count seats per model. Use GET(PARSE_JSON("model"),'en') as model_en. Count distinct seat_no? seats table has rows per seat. So COUNT(*) per aircraft_code then join model. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("model"), 'en')::VARCHAR AS "model", COUNT(*) AS "seat_count"
FROM AIRLINES.AIRLINES."SEATS"
JOIN AIRLINES.AIRLINES."AIRCRAFTS_DATA" ON "SEATS"."aircraft_code" = "AIRCRAFTS_DATA"."aircraft_code"
GROUP BY 1
ORDER BY 2 DESC

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (9 rows, 1.05s)

| model           |   seat_count |
|:----------------|-------------:|
| Boeing 777-300  |          402 |
| Boeing 767-300  |          222 |
| Airbus A321-200 |          170 |
| Airbus A320-200 |          140 |
| Boeing 737-300  |          130 |

 ---
🤖 **Assistant**

The aircraft models with the most seats available (based on the `SEATS` table) are:

| Model | Seats |
|-------|-------|
| Boeing 777‑300 | 402 |
| Boeing 767‑300 | 222 |
| Airbus A321‑200 | 170 |
| Airbus A320‑200 | 140 |
| Boeing 737‑300 | 130 |
| Airbus A319‑100 | 116 |
| Sukhoi Superjet‑100 | 97 |
| Bombardier CRJ‑200 | 50 |
| Cessna 208 Caravan | 12 |

These counts represent the total number of seat rows recorded for each aircraft model in the dataset.

💭 _Reasoning: Provide answer._


### 5

In [ ]:
q = "What is the average booking amount by all months?"
chat, res = ask(q)
res

```sql
SELECT DATE_TRUNC('month', TO_TIMESTAMP("book_date")) AS "month",
       AVG("total_amount") AS "average_booking_amount"
FROM AIRLINES.AIRLINES."BOOKINGS"
GROUP BY 1
ORDER BY 1;
```

**Result (first 10 rows)**  

| month                | average_booking_amount |
|----------------------|------------------------|
| 2017‑06‑01 00:00:00  | 76319.457757            |
| 2017‑07‑01 00:00:00  | 79217.606637            |
| 2017‑08‑01 00:00:00  | 78886.354229            |

(Only the first three months are shown; the query returns all months in the dataset.)

<details>

- id: `chatcmpl-mijy7ou6wld4b3u5jyvask`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=183, prompt_tokens=3345, total_tokens=3528, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
vis(chat)

### 👤 **User**

> What is the average booking amount by all months?

 ---
### 🧠 Assistant 


_Reasoning: Need avg total_amount per month from BOOKINGS.book_date. Extract month. Use to_timestamp? book_date string with timezone. Use TO_TIMESTAMP(book_date). Then extract month via DATE_TRUNC or EXTRACT(MONTH...). Let's use DATE_TRUNC('month', TO_TIMESTAMP("book_date")). Group by that. Return month and avg._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT DATE_TRUNC('month', TO_TIMESTAMP("book_date")) AS "month", AVG("total_amount") AS "average_booking_amount" FROM AIRLINES.AIRLINES."BOOKINGS" GROUP BY 1 ORDER BY 1

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (3 rows, 0.65s)

| month               |   average_booking_amount |
|:--------------------|-------------------------:|
| 2017-06-01 00:00:00 |                  76319.5 |
| 2017-07-01 00:00:00 |                  79217.6 |
| 2017-08-01 00:00:00 |                  78886.4 |

 ---
🤖 **Assistant**

```sql
SELECT DATE_TRUNC('month', TO_TIMESTAMP("book_date")) AS "month",
       AVG("total_amount") AS "average_booking_amount"
FROM AIRLINES.AIRLINES."BOOKINGS"
GROUP BY 1
ORDER BY 1;
```

**Result (first 10 rows)**  

| month                | average_booking_amount |
|----------------------|------------------------|
| 2017‑06‑01 00:00:00  | 76319.457757            |
| 2017‑07‑01 00:00:00  | 79217.606637            |
| 2017‑08‑01 00:00:00  | 78886.354229            |

(Only the first three months are shown; the query returns all months in the dataset.)

💭 _Reasoning: Need to output result._


In [ ]:
q = "Business class revenue wrt most used route"
chat, res = ask(q)
res

Here’s a concise query that lists the top routes by Business‑class revenue:

```sql
SELECT 
  "departure_airport" AS "departure",
  "arrival_airport"   AS "arrival",
  SUM("amount")       AS "business_revenue"
FROM AIRLINES.AIRLINES.TICKET_FLIGHTS
JOIN AIRLINES.AIRLINES.FLIGHTS
  ON AIRLINES.AIRLINES.TICKET_FLIGHTS."flight_id" = AIRLINES.AIRLINES.FLIGHTS."flight_id"
WHERE AIRLINES.AIRLINES.TICKET_FLIGHTS."fare_conditions" = 'Business'
GROUP BY "departure_airport", "arrival_airport"
ORDER BY "business_revenue" DESC
LIMIT 10;
```

This returns the most lucrative Business‑class routes, ranked by total revenue.

<details>

- id: `chatcmpl-2xxp5lmtjrc8pey3vkayy`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=199, prompt_tokens=3910, total_tokens=4109, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
vis(chat)

### 👤 **User**

> Business class revenue wrt most used route

 ---
### 🧠 Assistant 


_Reasoning: Need query: sum amount per route (departure_airport-arrival_airport) for business fare_conditions. Use TICKET_FLIGHTS join FLIGHTS to get airports. Sum amount where fare_conditions='Business'. Group by route. Order desc limit maybe 10._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("departure_airport"), 'en')::VARCHAR AS "departure", GET(PARSE_JSON("arrival_airport"), 'en')::VARCHAR AS "arrival", SUM("amount") AS "business_revenue" FROM AIRLINES.AIRLINES.TICKET_FLIGHTS JOIN AIRLINES.AIRLINES.FLIGHTS ON TICKET_FLIGHTS.flight_id = FLIGHTS.flight_id WHERE TICKET_FLIGHTS.fare_conditions = 'Business' GROUP BY 1,2 ORDER BY 3 DESC LIMIT 10

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

❌ **Error:** 000904 (42000): SQL compilation error: error line 1 at position 247
invalid identifier 'TICKET_FLIGHTS.FLIGHT_ID'
 ---
### 🧠 Assistant 


_Reasoning: Need fully qualified names and correct column references. Use AIRLINES.AIRLINES.TICKET_FLIGHTS etc. Also departure_airport is in FLIGHTS, not JSON. So no parse_json. Just use columns directly. Let's craft query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT "departure_airport" AS "departure", "arrival_airport" AS "arrival", SUM("amount") AS "business_revenue" FROM AIRLINES.AIRLINES.TICKET_FLIGHTS JOIN AIRLINES.AIRLINES.FLIGHTS ON AIRLINES.AIRLINES.TICKET_FLIGHTS."flight_id" = AIRLINES.AIRLINES.FLIGHTS."flight_id" WHERE AIRLINES.AIRLINES.TICKET_FLIGHTS."fare_conditions" = 'Business' GROUP BY "departure_airport", "arrival_airport" ORDER BY "business_revenue" DESC LIMIT 10

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (10 rows, 8.56s)

| departure   | arrival   |   business_revenue |
|:------------|:----------|-------------------:|
| DME         | KHV       |          237082500 |
| KHV         | DME       |          233577000 |
| KHV         | LED       |          159945800 |
| LED         | KHV       |          156408000 |
| LED         | IKT       |          151771800 |

 ---
🤖 **Assistant**

Here’s a concise query that lists the top routes by Business‑class revenue:

```sql
SELECT 
  "departure_airport" AS "departure",
  "arrival_airport"   AS "arrival",
  SUM("amount")       AS "business_revenue"
FROM AIRLINES.AIRLINES.TICKET_FLIGHTS
JOIN AIRLINES.AIRLINES.FLIGHTS
  ON AIRLINES.AIRLINES.TICKET_FLIGHTS."flight_id" = AIRLINES.AIRLINES.FLIGHTS."flight_id"
WHERE AIRLINES.AIRLINES.TICKET_FLIGHTS."fare_conditions" = 'Business'
GROUP BY "departure_airport", "arrival_airport"
ORDER BY "business_revenue" DESC
LIMIT 10;
```

This returns the most lucrative Business‑class routes, ranked by total revenue.

💭 _Reasoning: Need to format query with JSON extraction? departure_airport is plain code, not JSON. So fine. Provide final answer with triple quotes._


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()